# Benchmarking / Profiling

In [1]:
%load_ext autoreload
%autoreload 2

import collections
import ctypes
import itertools
import multiprocessing
import numpy as np
import matplotlib.pyplot as plt

import nerdle
import score as s
import generator
sgo = ctypes.CDLL(s.SCORE_GUESS_OPT_SO)
from nerdle import Hint

In [2]:
# Mini-Nerdle.
NUM_SLOTS = 6
SCORE_DB_FILE = "db/nerdle{}.db".format(NUM_SLOTS) 
solver_data = nerdle.create_solver_data(NUM_SLOTS, SCORE_DB_FILE)
d = solver_data.score_db
print(d.shape)

(206, 206)


## Solver

In [7]:
%timeit guess_history, hint_history, answer_size_history =  nerdle.NerdleSolver(solver_data).solve("4*3=12", initial_guess="56/7=8")

3.63 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Answer Generation

In [8]:
for num_slots in range(5, 9):
    print(num_slots)
    %time a = list(generator.all_answers(num_slots));
    print(len(a))

5
CPU times: user 2.7 ms, sys: 41 µs, total: 2.74 ms
Wall time: 2.78 ms
217
6
CPU times: user 49.1 ms, sys: 1.09 ms, total: 50.2 ms
Wall time: 49.9 ms
206
7
CPU times: user 635 ms, sys: 1.8 ms, total: 637 ms
Wall time: 638 ms
7561
8
CPU times: user 9.94 s, sys: 22.3 ms, total: 9.96 s
Wall time: 9.99 s
17723


## Score Database - Parallel Construction

In [5]:
answers = list(generator.all_answers(8))

In [8]:
n = 2000
a = answers[:n]
%time result = [nerdle._score_guess((guess, answer)) for guess in a for answer in a]

CPU times: user 2.64 s, sys: 39 ms, total: 2.67 s
Wall time: 2.68 s


In [9]:
# with multiprocessing.Pool(8) as pool:
#     %time result1 = pool.starmap(nerdle._score_guess, itertools.product(a, repeat=2))

In [13]:
# Slow.
# with multiprocessing.Pool(multiprocessing.cpu_count() // 2) as pool:
#     %time result2 = [pool.apply_async(nerdle._score_guess, t).get() for t in itertools.product(a, repeat=2)]
# Doesn't work yet.
# from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
# with ProcessPoolExecutor(max_workers=4) as pool:
#     %time result1 = pool.map(nerdle._score_guess, itertools.product(a, repeat=2))

In [10]:
# from test_parallel import square

# n = 10000000
# with multiprocessing.Pool(processes=4) as pool:
#     %time results = pool.map(square, itertools.repeat(3, n))

In [13]:
from test_parallel import process_one_arg

n = 10000000
print(n, n ** 0.5)
%time results = [process_one_arg("54/9=6") for _ in range(n)]
for p in (1, 2, 4, 8, 12):
    with multiprocessing.Pool(processes=p) as pool:
        %time results = pool.map(process_one_arg, itertools.repeat("54/9=6", n))

10000000 3162.2776601683795
CPU times: user 5.14 s, sys: 129 ms, total: 5.27 s
Wall time: 5.28 s
CPU times: user 859 ms, sys: 297 ms, total: 1.16 s
Wall time: 6.19 s
CPU times: user 793 ms, sys: 290 ms, total: 1.08 s
Wall time: 3.41 s
CPU times: user 852 ms, sys: 297 ms, total: 1.15 s
Wall time: 2.25 s
CPU times: user 1.04 s, sys: 331 ms, total: 1.37 s
Wall time: 2.03 s
CPU times: user 1.14 s, sys: 300 ms, total: 1.44 s
Wall time: 2.17 s


In [18]:
# from test_parallel import process_args

# n = 2000
# a = answers[:n]
# %time result = [[nerdle._score_guess(guess, answer) for answer in a] for guess in a]

# with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
#     %time result2 = pool.map(process_args, itertools.product(a, repeat=2))
# result == result2
# np.array_equal(np.array(result2).reshape(n, n), np.array(result))

In [14]:
num_slots = 8
# # Parallel implementation.
# %time solver_data = nerdle.create_solver_data(num_slots, "db/nerdle{}.db".format(num_slots), overwrite=True, min_parallel_n=len(answers) // 2)
# Serial implementation.
%time solver_data = nerdle.create_solver_data(num_slots, "db/nerdle{}.db".format(num_slots), overwrite=True, min_parallel_n=2 * len(answers))

0 / 17723 (0.0%) completed
886 / 17723 (5.0%) completed
1772 / 17723 (10.0%) completed
2658 / 17723 (15.0%) completed
3544 / 17723 (20.0%) completed
4430 / 17723 (25.0%) completed
5316 / 17723 (30.0%) completed
6202 / 17723 (35.0%) completed
7088 / 17723 (40.0%) completed
7974 / 17723 (45.0%) completed
8860 / 17723 (50.0%) completed
9746 / 17723 (55.0%) completed
10632 / 17723 (60.0%) completed
11518 / 17723 (65.0%) completed
12404 / 17723 (70.0%) completed
13290 / 17723 (75.0%) completed
14176 / 17723 (80.0%) completed
15062 / 17723 (85.0%) completed
15948 / 17723 (90.0%) completed
16834 / 17723 (95.0%) completed
17720 / 17723 (100.0%) completed
CPU times: user 3min 11s, sys: 30.9 s, total: 3min 42s
Wall time: 3min 46s
